In [56]:
import psycopg2 as pg
import json
import sys
import pandas.io.sql as psql
import plotly.plotly as py 

import numpy

try:
    conn = pg.connect(host='localhost')
except:
    print("I am unable to connect to the database")
    
cur = conn.cursor()
cur.execute("SET search_path = carvana")
    

In [57]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import plotly
plotly.tools.set_credentials_file(username='adervish', api_key='Ddss5qpHA8iN2IWuqDJs')

In [58]:
cvna_popular = psql.read_sql("""select make, model, count(1) from carvana_vehicles group by make, model order by count(1) desc;""", conn)
kmx_popular = psql.read_sql("""select make, model, count(1) from kmx_vehicles group by make, model order by count(1) desc;""", conn)

In [39]:
cvna_popular

,make,model,count
0,Honda,Civic,339
1,Hyundai,Sonata,211
2,Nissan,Altima,201
3,BMW,3 Series,181
4,Ford,Fusion,173
5,Hyundai,Elantra,162
6,Hyundai,Accent,160
7,Toyota,Corolla,145
8,Volkswagen,Passat,145
9,Toyota,Camry,141


In [60]:
df = psql.read_sql(""" SELECT c.year,
    c.miles AS c_miles,
    c.price AS c_price,
    kmx.price::text::double precision - c.price::text::double precision AS price_diff,
    kmx.price AS k_price,
    kmx.miles AS k_miles,
    c.vin, c.stockid,
    kmx.vin, kmx.stocknumber, kmx.transmission,
    c.make,
    c.model,
    c."trim" AS c_trim,
    kmx.descr AS k_trim,
    kmx.drivetrain AS k_drivetrain
   FROM carvana_vehicles c
     LEFT JOIN LATERAL ( SELECT k.miles, k.vin, k.stocknumber, k.transmission,
            k.drivetrain,
            k.price,
            k.make,
            k.model,
            k.descr
           FROM kmx_vehicles k
          WHERE k.model = c.model AND k.make = c.make AND c.year = k.year AND abs(k.miles - (c.miles / 1000::numeric)::integer::numeric) < 3::numeric AND k.descr::text ~~* (('%'::text || btrim(c."trim"::text, '"'::text)) || '"'::text) AND (k.drivetrain::text = '"2WD"'::text OR c.morefeatures ~~ (('%'::text || btrim(k.drivetrain::text, '"'::text)) || '%'::text))
          ORDER BY (abs(k.miles - (c.miles / 1000::numeric)::integer::numeric))
         LIMIT 1) kmx ON true where kmx.price is not null  """, conn)

In [50]:
data = [go.Histogram(x=df['price_diff'])]
py.iplot(data, filename='all price difference')

In [49]:
data = [go.Histogram(x=df[df.model == 'Civic']['price_diff'])]
py.iplot(data, filename='honda civic price difference')

In [51]:
data = [go.Histogram(x=df[df.model == 'Sonata']['price_diff'])]
py.iplot(data, filename='honda civic price difference')

In [ ]:
data = [go.Histogram(x=df[df.model == 'Elantra']['price_diff'])]
py.iplot(data, filename='honda civic price difference')

In [66]:
data = [go.Histogram(x=df[df.model == 'Fusion']['price_diff'])]
py.iplot(data, filename='honda civic price difference')

PlotlyRequestError: No message

In [24]:
dataframe['price_diff'].describe()

count     315.000000
mean     1129.460317
std       599.011229
min      -501.000000
25%       698.000000
50%      1198.000000
75%      1499.000000
max      2598.000000
Name: price_diff, dtype: float64